## Text Classification - Palm 2 Vertex AI
This is PaLM 2 model prompt designed for text classification. I complete this in a lab using VertexAI, This goes over different ways to implement text classification.

* Zero-shot prompting text classification
* Few-shot prompting text classification
* Common tasks:
    * Sentiment analysis
    * Topic classification
    * Spam detection
    * Intent recognition
    * Language identification
    * Toxicity detection
    * Emotion detection

In [4]:
 import vertexai

 PROJECT_ID = "[I REMOVED MY GOOGLE CLOD INFORMATION BUT ANYONE CAN INPUT THEIR OWN]"
 vertexai.init(project=PROJECT_ID, location="[I REMOVED MY GOOGLE CLOD INFORMATION BUT ANYONE CAN INPUT THEIR OWN]")

In [5]:
import pandas as pd
from vertexai.language_models import TextGenerationModel

### Import models

In [6]:
generation_model = TextGenerationModel.from_pretrained("text-bison@001")

## Text Classification

### Zero-shot prompting

Zero-shot prompting is where you do not provide examples with labels, and rely on the LLM to make the classification on its own.

In [7]:
prompt = """
Classify the following:\n
text: "I saw a furry animal in the park today with a long tail and big eyes."
label: dogs, cats
"""

print(
    generation_model.predict(
        prompt=prompt,
        max_output_tokens=256,
        temperature=0.1,
    ).text
)

The text is about a furry animal with a long tail and big eyes. It is likely to be a dog or a cat.


### Few-shot prompting

With few-shot prompting, you provide examples to the PaLM model and expect it to predict classes based on the provided examples.

In [8]:
prompt = """
What is the topic for a given news headline? \n
- business \n
- entertainment \n
- health \n
- sports \n
- technology \n\n

Text: Pixel 7 Pro Expert Hands On Review. \n
The answer is: technology \n

Text: Quit smoking? \n
The answer is: health \n

Text: Birdies or bogeys? Top 5 tips to hit under par \n
The answer is: sports \n

Text: Relief from local minimum-wage hike looking more remote \n
The answer is: business \n

Text: You won't guess who just arrived in Bari, Italy for the movie premiere. \n
The answer is:
"""

print(
    generation_model.predict(
        prompt=prompt,
        max_output_tokens=256,
        temperature=0.1,
    ).text
)

entertainment


### Other classification examples

Exploring some more common text classification prompts below, which are all based on zero-shot prompts.

#### Topic classification

In [9]:
prompt = """
Classify a piece of text into one of several predefined topics, such as sports, politics, or entertainment. \n
text: President Biden will be visiting India in the month of March to discuss a few opportunites. \n
class:
"""

print(
    generation_model.predict(
        prompt=prompt,
        max_output_tokens=256,
        temperature=0.1,
    ).text
)

politics


####  Spam detection

In [10]:
prompt = """
Given an email, classify it as spam or not spam. \n
email: hi user, \n
      you have been selected as a winner of the lotery and can win upto 1 million dollar. \n
      kindly share your bank details and we can proceed from there. \n\n

      from, \n
      US Official Lottry Depatmint
"""

print(
    generation_model.predict(
        prompt=prompt,
        max_output_tokens=256,
        temperature=0.1,
    ).text
)

spam


#### Intent recognition

In [11]:
prompt = """
Given a user's input, classify their intent, such as "finding information", "making a reservation", or "placing an order". \n
user input: Hi, can you please book a table for two at Juan for May 1?
"""

print(
    generation_model.predict(
        prompt=prompt,
        max_output_tokens=256,
        temperature=0.1,
    ).text
)

making a reservation


#### Language identification

In [12]:
prompt = """
Given a piece of text, classify the language it is written in. \n
text: Selam nasıl gidiyor?
language:
"""

print(
    generation_model.predict(
        prompt=prompt,
        max_output_tokens=256,
        temperature=0.1,
    ).text
)

Turkish


#### Toxicity detection

In [13]:
prompt = """
Given a piece of text, classify it as toxic or non-toxic. \n
text: i love sunny days
"""

print(
    generation_model.predict(
        prompt=prompt,
        max_output_tokens=256,
        temperature=0.1,
    ).text
)

Non-toxic


#### Emotion detection

In [14]:
prompt = """
Given a piece of text, classify the emotion it conveys, such as happiness, or anger. \n
text: I'm still so delighted from yesterday's news
"""

print(
    generation_model.predict(
        prompt=prompt,
        max_output_tokens=256,
        temperature=0.1,
    ).text
)

happiness


### Evaluation

Evaluated the outputs of the text classification task if the ground truth classes are available.

In [15]:
review_data = {
    "review": [
        "i love this product. it does have everything i am looking for!",
        "all i can say is that you will be happy after buying this product",
        "its way too expensive and not worth the price",
        "i am feeling okay. its neither good nor too bad.",
    ],
    "sentiment_groundtruth": ["positive", "positive", "negative", "neutral"],
}

review_data_df = pd.DataFrame(review_data)
review_data_df

,review,sentiment_groundtruth
0,i love this product. it does have everything i...,positive
1,all i can say is that you will be happy after ...,positive
2,its way too expensive and not worth the price,negative
3,i am feeling okay. its neither good nor too bad.,neutral


Each row will use the prompt in the `review` column to predict the sentiment using the PaLM API, and store the results in `sentiment_prediction` column.  

In [16]:
def get_sentiment(row):
    prompt = f"""Classify the sentiment of the following review as "positive", "neutral" and "negative". \n\n
                review: {row} \n
                sentiment:
              """
    response = generation_model.predict(prompt=prompt).text
    return response


review_data_df["sentiment_prediction"] = review_data_df["review"].apply(get_sentiment)
review_data_df

,review,sentiment_groundtruth,sentiment_prediction
0,i love this product. it does have everything i...,positive,positive
1,all i can say is that you will be happy after ...,positive,positive
2,its way too expensive and not worth the price,negative,negative
3,i am feeling okay. its neither good nor too bad.,neutral,neutral


The classification_report function shows how accurate the model was able to perform, and the model did very well.

In [17]:
from sklearn.metrics import classification_report

print(
    classification_report(
        review_data_df["sentiment_groundtruth"], review_data_df["sentiment_prediction"]
    )
)

              precision    recall  f1-score   support

    negative       1.00      1.00      1.00         1
     neutral       1.00      1.00      1.00         1
    positive       1.00      1.00      1.00         2

    accuracy                           1.00         4
   macro avg       1.00      1.00      1.00         4
weighted avg       1.00      1.00      1.00         4

